In [ ]:
! pip install catboost eli5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107720 sha256=697713f60b3679516501c0b5bfb136ea0cfab03e8deaa8998e22608c4a29adf3
  Stored in directory: /root/.cache/pip/wheels/b8/58/ef/2cf4c306898c2338d51540e0922c8e0d6028e07007085c0004
Successfully built eli5


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import re
from sklearn.preprocessing import StandardScaler
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import train_test_split
import eli5
from eli5.sklearn import PermutationImportance
from dateutil.relativedelta import relativedelta
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
import pickle
from xgboost import XGBClassifier
from hyperopt.early_stop import no_progress_loss
from catboost import CatBoostClassifier, Pool
from sklearn.linear_model import SGDClassifier
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 본인 환경에 맞게 경로 수정
path = '/content/drive/MyDrive/python/2024-1학기/MLOps/project/data/'
df = pd.read_csv(path+'data.csv', encoding= 'cp1252')

In [ ]:
class DropOutlier(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def is_numeric(self, text):
        return text.isdigit()

    def has_multiple_words(self, text):
        return len(text.split()) > 1

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.loc[X['InvoiceNo'].apply(self.is_numeric)]
        X = X.loc[(X['Quantity'] > 0) & (X['Quantity'] <= 100)]
        X.loc[:, 'Description'] = X.loc[:, 'Description'].astype(str)
        X = X.loc[(X['Description'].apply(self.has_multiple_words)) |
                  (X['Description'].isin(['SOMBRERO', 'POSTAGE', 'CARRIAGE']))]
        X = X.reset_index(drop=True)
        X = X.drop(columns=['StockCode']).drop_duplicates()
        return X


class FeatureEngineering(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X['InvoiceDate'] = pd.to_datetime(X['InvoiceDate'])
        X['InvoiceDate'] = X['InvoiceDate'].dt.date
        X = X.sort_values(by=['CustomerID', 'InvoiceDate'])
        X['Total_Price'] = X['Quantity'] * X['UnitPrice']
        X = X.groupby(['CustomerID', 'InvoiceDate']).agg({
            'Quantity': ['sum', 'min', 'max'],
            'UnitPrice': ['median', 'min', 'max'],
            'Total_Price': ['sum', 'mean', 'min', 'max']
        }).reset_index()
        X.columns = ['{}_{}'.format(col[0], col[1]) if col[1] else col[0] for col in X.columns]
        X['InvoiceDate'] = pd.to_datetime(X['InvoiceDate'])
        X['Quantity'] = X['Quantity_sum']
        X['Total_Price'] = X['Total_Price_sum']
        X['UnitPrice'] = X['UnitPrice_median']
        X.drop(columns=['Quantity_sum', 'Total_Price_sum', 'UnitPrice_median'], inplace=True)

        X['past_purchases'] = X.groupby('CustomerID').cumcount()
        X['past_purchase'] = np.where(X['past_purchases'] != 0, 1, X['past_purchases'])
        X['days_since_last_purchase'] = X.groupby('CustomerID')['InvoiceDate'].diff().dt.days
        X['days_since_last_purchase'] = X['days_since_last_purchase'].fillna(0).astype(int)
        X['days_since_last_purchase Expanding Mean'] = X.groupby(['CustomerID'])['days_since_last_purchase'].transform(
            lambda x: x.shift().expanding().mean()).fillna(0)
        X['first_purchase_date'] = X.groupby('CustomerID')['InvoiceDate'].transform('min')
        X['days_since_first_purchase'] = (X['InvoiceDate'] - X['first_purchase_date']).dt.days
        X.drop(columns='first_purchase_date', inplace=True)
        X['UnitPrice Expanding Mean'] = X.groupby(['CustomerID'])['UnitPrice'].transform(
            lambda x: x.shift().expanding().mean()).fillna(0)
        X['past_total_price'] = X.groupby('CustomerID')['Total_Price'].cumsum() - X['Total_Price']
        X['Total_Price Expanding Mean'] = X.groupby(['CustomerID'])['Total_Price'].transform(
            lambda x: x.shift().expanding().mean()).fillna(0)

        X['next_purchase_date'] = X.groupby('CustomerID')['InvoiceDate'].shift(-1)
        X['time_difference'] = (X['next_purchase_date'] - X['InvoiceDate']).dt.days
        X['Churn'] = np.where(X['time_difference'].isna(), np.nan, np.where(X['time_difference'] > 90, 1, 0))
        X.drop(columns=['time_difference', 'next_purchase_date'], inplace=True)
        X['time_difference'] = (X['InvoiceDate'].max() - X['InvoiceDate']).dt.days
        X['Churn'] = np.where((X.Churn.isna()) & (X['time_difference'] > 90), 1, X.Churn)
        X.drop(columns=['time_difference'], inplace=True)
        X['past_churns'] = X.groupby('CustomerID')['Churn'].cumsum() - X['Churn']
        X['past_churns'] = X.groupby('CustomerID')['past_churns'].ffill().fillna(0)
        X['past_churn'] = np.where(X['past_churns'] != 0, 1, X['past_churns'])
        X['churn_lag1'] = X.groupby('CustomerID')['Churn'].shift(1).fillna(0)

        X['UnitPrice_amplitude'] = X['UnitPrice'] - X['UnitPrice Expanding Mean']
        X['Total_Price_amplitude'] = X['Total_Price'] - X['Total_Price Expanding Mean']

        return X

class DropFeature(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X.drop(columns=['Quantity', 'past_churns', 'CustomerID'], inplace=True)

        return X


class CatPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, now_timestamp, churn_day=90, test_size=30, pred_size=7, foldnum=4, jump_day=30,hyperparameters=False):
        self.now_timestamp = now_timestamp
        self.churn_day = churn_day
        self.test_size = test_size
        self.pred_size = pred_size
        self.foldnum = foldnum
        self.jump_day = jump_day
        self.selected_features = None
        self.metrics_list = []
        self.all_fold_importances = []
        self.preprocessed_data = None
        self.hyperparameters = hyperparameters

        self.model = None
        self.train_pred = None
        self.test_pred = None

        self.final_model = None
        self.final_train_pred = None
        self.final_test_pred = None

    def make_train_test_pred(self, data, now_timestamp, churn_day=90, test_size=30, pred_size=7):
        test_timestamp = pd.Timestamp(now_timestamp) - relativedelta(days=churn_day)
        train_timestamp = pd.Timestamp(test_timestamp) - relativedelta(days=churn_day)
        train = data[data['InvoiceDate'] <= train_timestamp].sort_values(by='InvoiceDate')
        test = data[(data['InvoiceDate'] > (test_timestamp - relativedelta(days=test_size))) &
                    (data['InvoiceDate'] <= test_timestamp)].sort_values(by='InvoiceDate')
        pred = data[(data['InvoiceDate'] > pd.Timestamp(now_timestamp) - relativedelta(days=pred_size)) &
                    (data['InvoiceDate'] <= pd.Timestamp(now_timestamp))].sort_values(by='InvoiceDate')
        final_train = data[data['InvoiceDate'] <= test_timestamp].sort_values(by='InvoiceDate')
        return train, test, pred, final_train

    def make_validation_timestamp(self, data, now_timestamp, foldnum=4, jump_day=30):
        validation_timestamps = []
        now_timestamp = pd.Timestamp(now_timestamp)
        for i in range(foldnum):
            now_timestamp -= relativedelta(days=jump_day)
            validation_timestamps.append(now_timestamp.date().strftime('%Y-%m-%d'))
        return sorted(validation_timestamps)

    def calculate_permutation_importance(self, model, X_valid, y_valid):
        perm = PermutationImportance(model, scoring='f1', random_state=0)
        perm.fit(X_valid, y_valid)
        return perm.feature_importances_

    def preprocess(self, X):
        pipeline = Pipeline([
            ('drop_outlier', DropOutlier()),
            ('feature_engineering', FeatureEngineering()),
            ('drop_feature', DropFeature())
        ])
        self.preprocessed_data = pipeline.fit_transform(X)
        return self.preprocessed_data

    def preprocess_nodrop(self, X):
        pipeline = Pipeline([
            ('drop_outlier', DropOutlier()),
            ('feature_engineering', FeatureEngineering())

        ])
        self.preprocessed_data_nodrop = pipeline.fit_transform(X)
        return self.preprocessed_data_nodrop



    def select_features(self):
        fold_timestamp = self.make_validation_timestamp(self.preprocessed_data, self.now_timestamp, self.foldnum, self.jump_day)
        n_folds = self.foldnum
        fold_weights = np.arange(1, n_folds + 1)

        for timestamp in fold_timestamp:
            print('---', timestamp, 'fold----------------------')
            train, valid, _, _ = self.make_train_test_pred(self.preprocessed_data, timestamp, self.churn_day, self.test_size, self.pred_size)

            print('train 시작 시점 :', train.iloc[0]['InvoiceDate'])
            print('train 마지막 시점 :', train.iloc[-1]['InvoiceDate'])
            print('valid 시작 시점 :', valid.iloc[0]['InvoiceDate'])
            print('valid 마지막 시점 :', valid.iloc[-1]['InvoiceDate'])
            print('\n')

            X_train = train.drop(columns=['Churn', 'InvoiceDate'])
            y_train = train.Churn
            X_test = valid.drop(columns=['Churn', 'InvoiceDate'])
            y_test = valid.Churn

            feature_names = X_train.columns.to_list()
            cat_features = [i for i in range(len(X_train.columns)) if X_train.dtypes.iloc[i] == 'object']

            train_pool = Pool(data=X_train.values, label=y_train.values, feature_names=feature_names, cat_features=cat_features)
            test_pool = Pool(data=X_test.values, label=y_test.values, feature_names=feature_names, cat_features=cat_features)

            model = CatBoostClassifier(random_seed=0, iterations=100, eval_metric='F1', class_weights=[1, 10], verbose=0)
            model.fit(train_pool)

            y_pred = model.predict(test_pool)
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            cm = confusion_matrix(y_test, y_pred)

            self.metrics_list.append({'accuracy': accuracy, 'auc': auc, 'f1': f1, 'precision': precision})

            importances = self.calculate_permutation_importance(model, X_test, y_test)
            self.all_fold_importances.append(importances)

        total_weight = sum(fold_weights)
        weighted_avg = {
            'accuracy': sum(metrics['accuracy'] * weight for metrics, weight in zip(self.metrics_list, fold_weights)) / total_weight,
            'auc': sum(metrics['auc'] * weight for metrics, weight in zip(self.metrics_list, fold_weights)) / total_weight,
            'f1': sum(metrics['f1'] * weight for metrics, weight in zip(self.metrics_list, fold_weights)) / total_weight,
            'precision': sum(metrics['precision'] * weight for metrics, weight in zip(self.metrics_list, fold_weights)) / total_weight
        }

        weighted_importances = np.average(self.all_fold_importances, axis=0, weights=fold_weights)
        feature_importances = pd.Series(weighted_importances, index=feature_names).sort_values(ascending=False)
        self.selected_features = feature_importances.loc[feature_importances.values > 0].index.to_list()

        print("Weighted averages of validation sets (recent folds weighted more):")
        print(f"Weighted Accuracy: {weighted_avg['accuracy']:.4f}")
        print(f"Weighted AUC: {weighted_avg['auc']:.4f}")
        print(f"Weighted F1-score: {weighted_avg['f1']:.4f}")
        print(f"Weighted Precision: {weighted_avg['precision']:.4f}")
        print('\n')

        plt.figure(figsize=(6, 4))
        feature_importances.plot(kind='barh')
        plt.title('Weighted Permutation Importances')
        plt.xlabel('Features')
        plt.ylabel('Importance')
        plt.gca().invert_yaxis()
        plt.show()
        print('\n')



        return self.selected_features


    def train_with_selected_features(self):
        fold_timestamp = self.make_validation_timestamp(self.preprocessed_data, self.now_timestamp, self.foldnum, self.jump_day)
        n_folds = self.foldnum
        fold_weights = np.arange(1, n_folds + 1)
        new_metrics_list = []

        for timestamp in fold_timestamp:
            train, valid, _, _ = self.make_train_test_pred(self.preprocessed_data, timestamp, self.churn_day, self.test_size, self.pred_size)

            X_train = train[self.selected_features]
            y_train = train.Churn
            X_test = valid[self.selected_features]
            y_test = valid.Churn

            feature_names = X_train.columns.to_list()
            cat_features = [i for i in range(len(X_train.columns)) if X_train.dtypes.iloc[i] == 'object']

            train_pool = Pool(data=X_train.values, label=y_train.values, feature_names=feature_names, cat_features=cat_features)
            test_pool = Pool(data=X_test.values, label=y_test.values, feature_names=feature_names, cat_features=cat_features)

            model = CatBoostClassifier(random_seed=0, iterations=100, eval_metric='F1', class_weights=[1, 10], verbose=0)
            model.fit(train_pool)

            y_pred = model.predict(test_pool)
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)

            new_metrics_list.append({'accuracy': accuracy, 'auc': auc, 'f1': f1, 'precision': precision})

        total_weight = sum(fold_weights)
        final_weighted_avg = {
            'accuracy': sum(metrics['accuracy'] * weight for metrics, weight in zip(new_metrics_list, fold_weights)) / total_weight,
            'auc': sum(metrics['auc'] * weight for metrics, weight in zip(new_metrics_list, fold_weights)) / total_weight,
            'f1': sum(metrics['f1'] * weight for metrics, weight in zip(new_metrics_list, fold_weights)) / total_weight,
            'precision': sum(metrics['precision'] * weight for metrics, weight in zip(new_metrics_list, fold_weights)) / total_weight
        }

        self.final_metrics_0 = final_weighted_avg

        print("------Final weighted averages of validation sets with selected features (recent folds weighted more)------",'\n')

        print(f"Final Weighted Accuracy: {final_weighted_avg['accuracy']:.4f}")
        print(f"Final Weighted AUC: {final_weighted_avg['auc']:.4f}")
        print(f"Final Weighted F1-score: {final_weighted_avg['f1']:.4f}")
        print(f"Final Weighted Precision: {final_weighted_avg['precision']:.4f}")
        print('\n\n')

        return self.final_metrics_0

    def train_with_selected_hyperparameters(self):
        fold_timestamp = self.make_validation_timestamp(self.preprocessed_data, self.now_timestamp, self.foldnum, self.jump_day)
        n_folds = self.foldnum
        fold_weights = np.arange(1, n_folds + 1)
        new_metrics_list = []

        for timestamp in fold_timestamp:
            train, valid, _, _ = self.make_train_test_pred(self.preprocessed_data, timestamp, self.churn_day, self.test_size, self.pred_size)

            X_train = train[self.selected_features]
            y_train = train.Churn
            X_test = valid[self.selected_features]
            y_test = valid.Churn

            feature_names = X_train.columns.to_list()
            cat_features = [i for i in range(len(X_train.columns)) if X_train.dtypes.iloc[i] == 'object']

            train_pool = Pool(data=X_train.values, label=y_train.values, feature_names=feature_names, cat_features=cat_features)
            test_pool = Pool(data=X_test.values, label=y_test.values, feature_names=feature_names, cat_features=cat_features)

            model = CatBoostClassifier(random_seed=0, depth = 10, l2_leaf_reg = 20,random_strength=20, iterations=100, eval_metric='F1', class_weights=[1, 7], verbose=0)
            model.fit(train_pool)

            y_pred = model.predict(test_pool)
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)

            new_metrics_list.append({'accuracy': accuracy, 'auc': auc, 'f1': f1, 'precision': precision})

        total_weight = sum(fold_weights)
        final_weighted_avg = {
            'accuracy': sum(metrics['accuracy'] * weight for metrics, weight in zip(new_metrics_list, fold_weights)) / total_weight,
            'auc': sum(metrics['auc'] * weight for metrics, weight in zip(new_metrics_list, fold_weights)) / total_weight,
            'f1': sum(metrics['f1'] * weight for metrics, weight in zip(new_metrics_list, fold_weights)) / total_weight,
            'precision': sum(metrics['precision'] * weight for metrics, weight in zip(new_metrics_list, fold_weights)) / total_weight
        }

        self.final_metrics_1 = final_weighted_avg

        print("------Final weighted averages of validation sets with selected features and hyperparameters (recent folds weighted more)------",'\n')

        print(f"Final Weighted Accuracy: {final_weighted_avg['accuracy']:.4f}")
        print(f"Final Weighted AUC: {final_weighted_avg['auc']:.4f}")
        print(f"Final Weighted F1-score: {final_weighted_avg['f1']:.4f}")
        print(f"Final Weighted Precision: {final_weighted_avg['precision']:.4f}")

        return self.final_metrics_1

    def final_train(self):
        train, test, pred, final_train = self.make_train_test_pred(self.preprocessed_data, self.now_timestamp, self.churn_day, self.test_size, self.pred_size)
        print('\n')
        print('---------Final Model Train---------')
        print('\n')
        print('train 시작 시점 :', train.iloc[0]['InvoiceDate'])
        print('train 마지막 시점 :', train.iloc[-1]['InvoiceDate'])
        print('test 시작 시점 :', test.iloc[0]['InvoiceDate'])
        print('test 마지막 시점 :', test.iloc[-1]['InvoiceDate'])
        print('pred 시작 시점 :', pred.iloc[0]['InvoiceDate'])
        print('pred 마지막 시점 :', pred.iloc[-1]['InvoiceDate'])
        print('\n')


        if self.hyperparameters == False:
            X_train = train[self.selected_features]
            y_train = train.Churn
            X_test = test[self.selected_features]
            y_test = test.Churn

            feature_names = X_train.columns.to_list()
            cat_features = [i for i in range(len(X_train.columns)) if X_train.dtypes.iloc[i] == 'object']

            train_pool = Pool(data=X_train.values, label=y_train.values, feature_names=feature_names, cat_features=cat_features)
            test_pool = Pool(data=X_test.values, label=y_test.values, feature_names=feature_names, cat_features=cat_features)

            model = CatBoostClassifier(random_seed=0, iterations=100, eval_metric='F1', class_weights=[1, 10], verbose=0)
            model.fit(train_pool)

            y_pred = model.predict(test_pool)
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)

            print(f"Final Test Accuracy: { accuracy:.4f}")
            print(f"Final Test AUC: {auc:.4f}")
            print(f"Final Test F1-score: {f1:.4f}")
            print(f"Final Test Precision: {precision:.4f}")

        if self.hyperparameters == True:
          if self.final_metrics_0['f1'] < self.final_metrics_1['f1']:

            X_train = train[self.selected_features]
            y_train = train.Churn
            X_test = test[self.selected_features]
            y_test = test.Churn

            feature_names = X_train.columns.to_list()
            cat_features = [i for i in range(len(X_train.columns)) if X_train.dtypes.iloc[i] == 'object']

            train_pool = Pool(data=X_train.values, label=y_train.values, feature_names=feature_names, cat_features=cat_features)
            test_pool = Pool(data=X_test.values, label=y_test.values, feature_names=feature_names, cat_features=cat_features)

            model = CatBoostClassifier(random_seed=0, depth = 10, l2_leaf_reg = 20,random_strength=20, iterations=100, eval_metric='F1', class_weights=[1, 7], verbose=0)
            model.fit(train_pool)

            y_pred = model.predict(test_pool)
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)

            print(f"Final Test Accuracy: { accuracy:.4f}")
            print(f"Final Test AUC: {auc:.4f}")
            print(f"Final Test F1-score: {f1:.4f}")
            print(f"Final Test Precision: {precision:.4f}")

          else:
            X_train = train[self.selected_features]
            y_train = train.Churn
            X_test = test[self.selected_features]
            y_test = test.Churn

            feature_names = X_train.columns.to_list()
            cat_features = [i for i in range(len(X_train.columns)) if X_train.dtypes.iloc[i] == 'object']

            train_pool = Pool(data=X_train.values, label=y_train.values, feature_names=feature_names, cat_features=cat_features)
            test_pool = Pool(data=X_test.values, label=y_test.values, feature_names=feature_names, cat_features=cat_features)

            model = CatBoostClassifier(random_seed=0, iterations=100, eval_metric='F1', class_weights=[1, 10], verbose=0)
            model.fit(train_pool)

            y_pred = model.predict(test_pool)
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)

            print(f"Final Test Accuracy: { accuracy:.4f}")
            print(f"Final Test AUC: {auc:.4f}")
            print(f"Final Test F1-score: {f1:.4f}")
            print(f"Final Test Precision: {precision:.4f}")

        self.model = model
        self.train_pred =  model.predict_proba(train_pool)
        self.test_pred = model.predict_proba(test_pool)

        X_tmp = final_train[self.selected_features]
        y_tmp = final_train.Churn

        X_pred = pred[self.selected_features]


        feature_names = X_tmp.columns.to_list()
        cat_features = [i for i in range(len(X_tmp.columns)) if X_tmp.dtypes.iloc[i] == 'object']

        tmp_pool = Pool(data=X_tmp.values, label=y_tmp.values, feature_names=feature_names, cat_features=cat_features)
        pred_pool = Pool(data=X_pred.values, feature_names=feature_names, cat_features=cat_features)

        # model = CatBoostClassifier(random_seed=0, iterations=100, eval_metric='F1', class_weights=[1, 10], verbose=0)
        model.fit(tmp_pool)

        self.final_model = model
        self.final_train_pred = model.predict_proba(tmp_pool)
        self.final_test_pred = model.predict_proba(pred_pool)

    def fit(self, X, y=None):
        self.preprocess(X)
        self.select_features()
        self.train_with_selected_features()
        if self.hyperparameters == True:
            self.train_with_selected_hyperparameters()
        # self.train_with_selected_hyperparameters()
        self.final_train()

        return self

    def transform(self, X, y=None):
        if self.preprocessed_data is None or self.selected_features is None:
            raise ValueError("The model needs to be fitted before calling transform.")
        return self.preprocessed_data[self.selected_features]


In [ ]:
class ElaPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, now_timestamp, churn_day=90, test_size=30, pred_size=7, foldnum=4, jump_day=30, hyperparameters_fixed=True, hyperparameters=None):
        self.now_timestamp = now_timestamp
        self.churn_day = churn_day
        self.test_size = test_size
        self.pred_size = pred_size
        self.foldnum = foldnum
        self.jump_day = jump_day
        self.selected_features = None
        self.metrics_list = []
        self.all_fold_importances = []
        self.preprocessed_data = None
        self.hyperparameters_fixed = hyperparameters_fixed
        self.hyperparameters = hyperparameters
        self.final_model = None
        self.final_model_r = None
        self.train_pred = None
        self.train_pred_r = None
        self.scaler = StandardScaler()
        self.best_fixed_f1 = -1
        self.best_user_f1 = -1
        self.best_hyperparams = {}

    def make_train_test_pred(self, data, now_timestamp, churn_day=90, test_size=30, pred_size=7):
        test_timestamp = pd.Timestamp(now_timestamp) - relativedelta(days=churn_day)
        train_timestamp = pd.Timestamp(test_timestamp) - relativedelta(days=churn_day)
        train = data[data['InvoiceDate'] <= train_timestamp].sort_values(by='InvoiceDate')
        test = data[(data['InvoiceDate'] > (test_timestamp - relativedelta(days=test_size))) &
                    (data['InvoiceDate'] <= test_timestamp)].sort_values(by='InvoiceDate')
        pred = data[(data['InvoiceDate'] > pd.Timestamp(now_timestamp) - relativedelta(days=pred_size)) &
                    (data['InvoiceDate'] <= pd.Timestamp(now_timestamp))].sort_values(by='InvoiceDate')

        final_test = data[data['InvoiceDate'] <= test_timestamp].sort_values(by='InvoiceDate')
        return train, test, pred, final_test

    def make_validation_timestamp(self, data, now_timestamp, foldnum=4, jump_day=30):
        validation_timestamps = []
        now_timestamp = pd.Timestamp(now_timestamp)
        for i in range(foldnum):
            now_timestamp -= relativedelta(days=jump_day)
            validation_timestamps.append(now_timestamp.date().strftime('%Y-%m-%d'))
        return sorted(validation_timestamps)

    def preprocess(self, X):
        pipeline = Pipeline([
            ('drop_outlier', DropOutlier()),
            ('feature_engineering', FeatureEngineering()),
            ('drop_feature', DropFeature())
        ])
        self.preprocessed_data = pipeline.fit_transform(X)
        return self.preprocessed_data

    def preprocess_nodrop(self, X):
        pipeline = Pipeline([
            ('drop_outlier', DropOutlier()),
            ('feature_engineering', FeatureEngineering())

        ])
        self.preprocessed_data_nodrop = pipeline.fit_transform(X)
        return self.preprocessed_data_nodrop

    def evaluate(self, y_test, y_pred):
        accuracy = accuracy_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        cm = confusion_matrix(y_test, y_pred)
        print(f"\nAccuracy on test set: {accuracy:.4f}, AUC on test set: {auc:.4f}")
        print(f"F1-score on test set: {f1:.4f}")
        print(f"Precision on test set: {precision:.4f}\n")
        print(cm)
        return accuracy, auc, f1, precision, cm

    def ElasticNet_cv(self):
        fold_timestamp = self.make_validation_timestamp(self.preprocessed_data, self.now_timestamp, self.foldnum, self.jump_day)
        n_folds = self.foldnum
        fold_weights = np.arange(1, n_folds + 1)

        for timestamp in fold_timestamp:
            print('---', timestamp, 'fold----------------------')
            train, valid, _ , _ = self.make_train_test_pred(self.preprocessed_data, timestamp, self.churn_day, self.test_size, self.pred_size)

            print('train 시작 시점 :', train.iloc[0]['InvoiceDate'])
            print('train 마지막 시점 :', train.iloc[-1]['InvoiceDate'])
            print('valid 시작 시점 :', valid.iloc[0]['InvoiceDate'])
            print('valid 마지막 시점 :', valid.iloc[-1]['InvoiceDate'])
            print('\n')

            X_train = train.drop(columns=['Churn', 'InvoiceDate'])
            X_train_sc = self.scaler.fit_transform(X_train)
            y_train = train.Churn
            X_valid = valid.drop(columns=['Churn', 'InvoiceDate'])
            X_valid_sc = self.scaler.transform(X_valid)
            y_valid = valid.Churn

            # Fixed hyperparameters
            fixed_model = SGDClassifier(penalty='elasticnet', alpha = 0.05, l1_ratio=0.2, class_weight={0: 1, 1: 4}, max_iter=1000, random_state=1,
                                        loss = 'log_loss')
            fixed_model.fit(X_train_sc, y_train)
            y_valid_pred = fixed_model.predict(X_valid_sc)
            accuracy, auc, f1, precision, cm = self.evaluate(y_valid, y_valid_pred)
            self.best_fixed_f1 = max(self.best_fixed_f1, f1)

            if not self.hyperparameters_fixed and self.hyperparameters:
                user_model = SGDClassifier(penalty='elasticnet', loss = 'log_loss', **self.hyperparameters, random_state=1)
                user_model.fit(X_train_sc, y_train)
                y_valid_pred = user_model.predict(X_valid_sc)
                accuracy, auc, f1, precision, cm = self.evaluate(y_valid, y_valid_pred)
                self.best_user_f1 = max(self.best_user_f1, f1)

                if f1 > self.best_user_f1:
                    self.best_user_f1 = f1
                    self.best_hyperparams = self.hyperparameters

            self.metrics_list.append({'accuracy': accuracy, 'auc': auc, 'f1': f1, 'precision': precision})
            coefficients = pd.DataFrame(fixed_model.coef_, columns=X_train.columns)
            print(f'회귀계수 0인 변수: {coefficients.columns[coefficients.eq(0).all()].tolist()}')

        total_weight = sum(fold_weights)
        weighted_avg = {
            'accuracy': sum(metrics['accuracy'] * weight for metrics, weight in zip(self.metrics_list, fold_weights)) / total_weight,
            'auc': sum(metrics['auc'] * weight for metrics, weight in zip(self.metrics_list, fold_weights)) / total_weight,
            'f1': sum(metrics['f1'] * weight for metrics, weight in zip(self.metrics_list, fold_weights)) / total_weight,
            'precision': sum(metrics['precision'] * weight for metrics, weight in zip(self.metrics_list, fold_weights)) / total_weight
        }

        self.final_metrics_0 = weighted_avg
        print("Weighted averages of validation sets (recent folds weighted more):")
        print(f"Weighted Accuracy: {weighted_avg['accuracy']:.4f}")
        print(f"Weighted AUC: {weighted_avg['auc']:.4f}")
        print(f"Weighted F1-score: {weighted_avg['f1']:.4f}")
        print(f"Weighted Precision: {weighted_avg['precision']:.4f}")
        print('\n')

        return self.final_metrics_0

    def final_train(self):
        train, test, pred, final_test = self.make_train_test_pred(self.preprocessed_data, self.now_timestamp, self.churn_day, self.test_size, self.pred_size)
        print('\n')
        print('---------Final Model Train---------')
        print('\n')
        print('train 시작 시점 :', train.iloc[0]['InvoiceDate'])
        print('train 마지막 시점 :', train.iloc[-1]['InvoiceDate'])
        print('test 시작 시점 :', test.iloc[0]['InvoiceDate'])
        print('test 마지막 시점 :', test.iloc[-1]['InvoiceDate'])
        print('pred 시작 시점 :', pred.iloc[0]['InvoiceDate'])
        print('pred 마지막 시점 :', pred.iloc[-1]['InvoiceDate'])
        print('\n')

        X_train = train.drop(columns=['Churn', 'InvoiceDate'])
        X_train_sc = self.scaler.fit_transform(X_train)
        y_train = train.Churn
        X_test = test.drop(columns=['Churn', 'InvoiceDate'])
        X_test_sc = self.scaler.transform(X_test)
        y_test = test.Churn
        X_pred_f = pred.drop(columns=['Churn', 'InvoiceDate'])
        X_pred_f_sc = self.scaler.transform(X_pred_f)

        X_final_test = final_test.drop(columns=['Churn', 'InvoiceDate'])
        X_final_test_sc = self.scaler.transform(X_final_test)
        y_final_test = final_test.Churn

        # 6/3까지 train, 8월 예측 모델
        # Determine which hyperparameters to use for the final model
        if self.best_fixed_f1 >= self.best_user_f1 or self.hyperparameters_fixed:  # 기존 하이퍼파라미터의 f1이 더 높은 경우는 기존 하이퍼파라미터 사용
            print("Using fixed hyperparameters for the final model.")
            final_model = SGDClassifier(penalty='elasticnet', alpha = 0.05, l1_ratio=0.2, class_weight={0: 1, 1: 4},
                                        loss = 'log_loss', max_iter=1000, random_state=1)
        else:
            print("Using user-specified hyperparameters for the final model.")
            final_model = SGDClassifier(penalty='elasticnet', loss = 'log_loss', **self.best_hyperparams, random_state=1)

        final_model.fit(X_train_sc, y_train) # 6/3까지 train
        y_test_pred = final_model.predict(X_test_sc) # 8월 예측
        self.train_pred = final_model.predict_proba(X_test_sc)
        accuracy, auc, f1, precision, cm = self.evaluate(y_test, y_test_pred) # 8월 예측 성능 확인

        self.final_model = final_model

        # 9월까지 train, 11월 예측 모델
        # Determine which hyperparameters to use for the final model


        if self.best_fixed_f1 >= self.best_user_f1 or self.hyperparameters_fixed:  # 기존 하이퍼파라미터의 f1이 더 높은 경우는 기존 하이퍼파라미터 사용
            print("Using fixed hyperparameters for the final model.")
            final_model_r = SGDClassifier(penalty='elasticnet', alpha = 0.05, l1_ratio=0.2, class_weight={0: 1, 1: 4},
                                             loss = 'log_loss', max_iter=1000, random_state=1)
        else:
            print("Using user-specified hyperparameters for the final model.")
            final_model_r = SGDClassifier(penalty='elasticnet', loss = 'log_loss', **self.best_hyperparams, random_state=1)

        final_model_r.fit(X_final_test_sc, y_final_test) # 9월까지 train
        y_pred_f = final_model_r.predict(X_pred_f_sc) # 11월 예측
        self.train_pred_r = final_model_r.predict_proba(X_pred_f_sc)

        self.final_model_r = final_model_r

        coefficients = pd.DataFrame(final_model_r.coef_, columns=X_train.columns)
        print(f'회귀계수 0인 변수: {coefficients.columns[coefficients.eq(0).all()].tolist()}')

        print(f"Final Test Accuracy: {accuracy:.4f}")
        print(f"Final Test AUC: {auc:.4f}")
        print(f"Final Test F1-score: {f1:.4f}")
        print(f"Final Test Precision: {precision:.4f}")


        # Save final model for now_timestamp specified
        model_pickle = f'final_model_{self.now_timestamp}.pkl'
        with open(model_pickle, 'wb') as model_pickle:
            pickle.dump(self.final_model, model_pickle)
        print(f"Final model saved as {model_pickle}")

    def fit(self, X, y=None):
        self.preprocess(X)
        self.ElasticNet_cv()
        self.final_train()
        return self


    def transform(self, X, y=None):
        if self.preprocessed_data is None or self.selected_features is None:
            raise ValueError("The model needs to be fitted before calling transform.")
        return self.preprocessed_data[self.selected_features]

In [ ]:
class XGBPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, now_timestamp, churn_day=90, test_size=30, pred_size=7, foldnum=4, jump_day=30,hyperparameters=False):
        self.now_timestamp = now_timestamp
        self.churn_day = churn_day
        self.test_size = test_size
        self.pred_size = pred_size
        self.foldnum = foldnum
        self.jump_day = jump_day
        self.selected_features = None
        self.metrics_list = []
        self.all_fold_importances = []
        self.preprocessed_data = None
        self.preprocessed_data_nodrop = None
        self.hyperparameters = hyperparameters
        self.final_model = None
        self.train_pred = None

        self.model = None
        self.train_pred = None
        self.test_pred = None

        self.final_model = None
        self.final_train_pred = None
        self.final_test_pred = None

    def make_train_test_pred(self, data, now_timestamp, churn_day=90, test_size=30, pred_size=7):
        test_timestamp = pd.Timestamp(now_timestamp) - relativedelta(days=churn_day)
        train_timestamp = pd.Timestamp(test_timestamp) - relativedelta(days=churn_day)
        train = data[data['InvoiceDate'] <= train_timestamp].sort_values(by='InvoiceDate')
        test = data[(data['InvoiceDate'] > (test_timestamp - relativedelta(days=test_size))) &
                    (data['InvoiceDate'] <= test_timestamp)].sort_values(by='InvoiceDate')
        pred = data[(data['InvoiceDate'] > pd.Timestamp(now_timestamp) - relativedelta(days=pred_size)) &
                    (data['InvoiceDate'] <= pd.Timestamp(now_timestamp))].sort_values(by='InvoiceDate')
        final_train = data[data['InvoiceDate'] <= test_timestamp].sort_values(by='InvoiceDate')
        return train, test, pred, final_train

    def make_validation_timestamp(self, data, now_timestamp, foldnum=4, jump_day=30):
        validation_timestamps = []
        now_timestamp = pd.Timestamp(now_timestamp)
        for i in range(foldnum):
            now_timestamp -= relativedelta(days=jump_day)
            validation_timestamps.append(now_timestamp.date().strftime('%Y-%m-%d'))
        return sorted(validation_timestamps)

    def calculate_permutation_importance(self, model, X_valid, y_valid):
        perm = PermutationImportance(model, scoring='f1', random_state=0)
        perm.fit(X_valid, y_valid)
        return perm.feature_importances_

    def preprocess(self, X):
        pipeline = Pipeline([
            ('drop_outlier', DropOutlier()),
            ('feature_engineering', FeatureEngineering()),
            ('drop_feature', DropFeature())
        ])
        self.preprocessed_data = pipeline.fit_transform(X)
        return self.preprocessed_data


    def preprocess_nodrop(self, X):
        pipeline = Pipeline([
            ('drop_outlier', DropOutlier()),
            ('feature_engineering', FeatureEngineering())

        ])
        self.preprocessed_data_nodrop = pipeline.fit_transform(X)
        return self.preprocessed_data_nodrop

    def select_features(self):
        fold_timestamp = self.make_validation_timestamp(self.preprocessed_data, self.now_timestamp, self.foldnum, self.jump_day)
        n_folds = self.foldnum
        fold_weights = np.arange(1, n_folds + 1)

        for timestamp in fold_timestamp:
            train, valid, _, _ = self.make_train_test_pred(self.preprocessed_data, timestamp, self.churn_day, self.test_size, self.pred_size)

            X_train = train.drop(columns=['Churn', 'InvoiceDate'])
            y_train = train.Churn
            X_test = valid.drop(columns=['Churn', 'InvoiceDate'])
            y_test = valid.Churn

            feature_names = X_train.columns.to_list()

            model = XGBClassifier(objective='binary:logistic', eval_metric='auc', random_state=0, use_label_encoder=False, scale_pos_weight = 10)
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)

            self.metrics_list.append({'accuracy': accuracy, 'auc': auc, 'f1': f1, 'precision': precision})

            importances = self.calculate_permutation_importance(model, X_test, y_test)
            self.all_fold_importances.append(importances)

        total_weight = sum(fold_weights)
        weighted_avg = {
            'accuracy': sum(metrics['accuracy'] * weight for metrics, weight in zip(self.metrics_list, fold_weights)) / total_weight,
            'auc': sum(metrics['auc'] * weight for metrics, weight in zip(self.metrics_list, fold_weights)) / total_weight,
            'f1': sum(metrics['f1'] * weight for metrics, weight in zip(self.metrics_list, fold_weights)) / total_weight,
            'precision': sum(metrics['precision'] * weight for metrics, weight in zip(self.metrics_list, fold_weights)) / total_weight
        }

        weighted_importances = np.average(self.all_fold_importances, axis=0, weights=fold_weights)
        feature_importances = pd.Series(weighted_importances, index=feature_names).sort_values(ascending=False)
        self.selected_features = feature_importances.loc[feature_importances.values > 0.001].index.to_list()

        print("Weighted averages of validation sets (recent folds weighted more):")
        print(f"Weighted Accuracy: {weighted_avg['accuracy']:.4f}")
        print(f"Weighted AUC: {weighted_avg['auc']:.4f}")
        print(f"Weighted F1-score: {weighted_avg['f1']:.4f}")
        print(f"Weighted Precision: {weighted_avg['precision']:.4f}")
        print('\n')

        plt.figure(figsize=(6, 4))
        feature_importances.plot(kind='barh')
        plt.title('Weighted Permutation Importances')
        plt.xlabel('Features')
        plt.ylabel('Importance')
        plt.gca().invert_yaxis()
        plt.show()
        print('\n')

        return self.selected_features


    def train_with_selected_features(self):
        fold_timestamp = self.make_validation_timestamp(self.preprocessed_data, self.now_timestamp, self.foldnum, self.jump_day)
        n_folds = self.foldnum
        fold_weights = np.arange(1, n_folds + 1)
        new_metrics_list = []

        for timestamp in fold_timestamp:
            train, valid, _, _ = self.make_train_test_pred(self.preprocessed_data, timestamp, self.churn_day, self.test_size, self.pred_size)

            X_train = train[self.selected_features]
            y_train = train.Churn
            X_test = valid[self.selected_features]
            y_test = valid.Churn

            model = XGBClassifier(objective='binary:logistic', eval_metric='auc', random_state=0, use_label_encoder=False, scale_pos_weight = 10)
            model.fit(X_train, y_train)

            feature_names = X_train.columns.to_list()

            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)

            new_metrics_list.append({'accuracy': accuracy, 'auc': auc, 'f1': f1, 'precision': precision})

        total_weight = sum(fold_weights)
        final_weighted_avg = {
            'accuracy': sum(metrics['accuracy'] * weight for metrics, weight in zip(new_metrics_list, fold_weights)) / total_weight,
            'auc': sum(metrics['auc'] * weight for metrics, weight in zip(new_metrics_list, fold_weights)) / total_weight,
            'f1': sum(metrics['f1'] * weight for metrics, weight in zip(new_metrics_list, fold_weights)) / total_weight,
            'precision': sum(metrics['precision'] * weight for metrics, weight in zip(new_metrics_list, fold_weights)) / total_weight
        }

        self.final_metrics_0 = final_weighted_avg

        print("------Final weighted averages of validation sets with selected features (recent folds weighted more)------",'\n')

        print(f"Final Weighted Accuracy: {final_weighted_avg['accuracy']:.4f}")
        print(f"Final Weighted AUC: {final_weighted_avg['auc']:.4f}")
        print(f"Final Weighted F1-score: {final_weighted_avg['f1']:.4f}")
        print(f"Final Weighted Precision: {final_weighted_avg['precision']:.4f}")
        print('\n\n')

        return self.final_metrics_0

    def train_with_selected_hyperparameters(self):
        fold_timestamp = self.make_validation_timestamp(self.preprocessed_data, self.now_timestamp, self.foldnum, self.jump_day)
        n_folds = self.foldnum
        fold_weights = np.arange(1, n_folds + 1)
        new_metrics_list = []

        for timestamp in fold_timestamp:
            train, valid, _, _ = self.make_train_test_pred(self.preprocessed_data, timestamp, self.churn_day, self.test_size, self.pred_size)

            X_train = train[self.selected_features]
            y_train = train.Churn
            X_test = valid[self.selected_features]
            y_test = valid.Churn

            model = XGBClassifier(
                colsample_bytree = 0.6210258338123061,
                learning_rate = 0.01173198307402594,
                max_depth = 6,
                n_estimators = 180,
                scale_pos_weight = 8.305813106322233,
                subsample = 0.16886950954669824,
                objective = 'binary:logistic',
                random_state = 0)
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)

            new_metrics_list.append({'accuracy': accuracy, 'auc': auc, 'f1': f1, 'precision': precision})

        total_weight = sum(fold_weights)
        final_weighted_avg = {
            'accuracy': sum(metrics['accuracy'] * weight for metrics, weight in zip(new_metrics_list, fold_weights)) / total_weight,
            'auc': sum(metrics['auc'] * weight for metrics, weight in zip(new_metrics_list, fold_weights)) / total_weight,
            'f1': sum(metrics['f1'] * weight for metrics, weight in zip(new_metrics_list, fold_weights)) / total_weight,
            'precision': sum(metrics['precision'] * weight for metrics, weight in zip(new_metrics_list, fold_weights)) / total_weight
        }

        self.final_metrics_1 = final_weighted_avg

        print("------Final weighted averages of validation sets with selected features and hyperparameters (recent folds weighted more)------",'\n')

        print(f"Final Weighted Accuracy: {final_weighted_avg['accuracy']:.4f}")
        print(f"Final Weighted AUC: {final_weighted_avg['auc']:.4f}")
        print(f"Final Weighted F1-score: {final_weighted_avg['f1']:.4f}")
        print(f"Final Weighted Precision: {final_weighted_avg['precision']:.4f}")

        return self.final_metrics_1

    def final_train(self):
        train, test, pred, final_train = self.make_train_test_pred(self.preprocessed_data, self.now_timestamp, self.churn_day, self.test_size, self.pred_size)
        print('\n')
        print('---------Final Model Train---------')
        print('\n')
        print('train 시작 시점 :', train.iloc[0]['InvoiceDate'])
        print('train 마지막 시점 :', train.iloc[-1]['InvoiceDate'])
        print('test 시작 시점 :', test.iloc[0]['InvoiceDate'])
        print('test 마지막 시점 :', test.iloc[-1]['InvoiceDate'])
        print('pred 시작 시점 :', pred.iloc[0]['InvoiceDate'])
        print('pred 마지막 시점 :', pred.iloc[-1]['InvoiceDate'])
        print('\n')


        if self.hyperparameters == False:
            X_train = train[self.selected_features]
            y_train = train.Churn
            X_test = test[self.selected_features]
            y_test = test.Churn

            feature_names = X_train.columns.to_list()
            model = XGBClassifier(objective='binary:logistic', eval_metric='auc', random_state=0, use_label_encoder=False, scale_pos_weight = 10)
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)

            print(f"Final Accuracy: { accuracy:.4f}")
            print(f"Final AUC: {auc:.4f}")
            print(f"Final F1-score: {f1:.4f}")
            print(f"Final Precision: {precision:.4f}")

        if self.hyperparameters == True:
          if self.final_metrics_0['f1'] < self.final_metrics_1['f1']:

            X_train = train[self.selected_features]
            y_train = train.Churn
            X_test = test[self.selected_features]
            y_test = test.Churn

            feature_names = X_train.columns.to_list()

            model = XGBClassifier(
                colsample_bytree = 0.6210258338123061,
                learning_rate = 0.01173198307402594,
                max_depth = 6,
                n_estimators = 180,
                scale_pos_weight = 8.305813106322233,
                subsample = 0.16886950954669824,
                objective = 'binary:logistic',
                random_state = 0)
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)

            print(f"Final Accuracy: { accuracy:.4f}")
            print(f"Final AUC: {auc:.4f}")
            print(f"Final F1-score: {f1:.4f}")
            print(f"Final Precision: {precision:.4f}")

          else:
            X_train = train[self.selected_features]
            y_train = train.Churn
            X_test = test[self.selected_features]
            y_test = test.Churn

            feature_names = X_train.columns.to_list()

            model = XGBClassifier(objective='binary:logistic', eval_metric='auc', random_state=0, use_label_encoder=False, scale_pos_weight = 10)
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)

            print(f"Final Test Accuracy: { accuracy:.4f}")
            print(f"Final Test AUC: {auc:.4f}")
            print(f"Final Test F1-score: {f1:.4f}")
            print(f"Final Test Precision: {precision:.4f}")

        self.model = model
        self.train_pred =  model.predict_proba(X_train)
        self.test_pred = model.predict_proba(X_test)

        X_tmp = final_train[self.selected_features]
        y_tmp = final_train.Churn

        X_pred = pred[self.selected_features]

        feature_names = X_tmp.columns.to_list()
        model.fit(X_tmp, y_tmp)

        self.final_model = model
        self.final_train_pred = model.predict_proba(X_tmp)
        self.final_test_pred = model.predict_proba(X_pred)

    def fit(self, X, y=None):
        self.preprocess(X)
        self.select_features()
        self.train_with_selected_features()
        if self.hyperparameters == True:
            self.train_with_selected_hyperparameters()
        # self.train_with_selected_hyperparameters()
        self.final_train()

        return self

    def transform(self, X, y=None):
        if self.preprocessed_data is None or self.selected_features is None:
            raise ValueError("The model needs to be fitted before calling transform.")
        return self.preprocessed_data[self.selected_features]

In [ ]:
class EnsemblePipeline(BaseEstimator, TransformerMixin):
    def __init__(self, now_timestamp, space, churn_day=90, test_size=30, pred_size=7, foldnum=4, jump_day=30, hyperparameters=False):
        self.now_timestamp = now_timestamp
        self.churn_day = churn_day
        self.test_size = test_size
        self.pred_size = pred_size
        self.foldnum = foldnum
        self.jump_day = jump_day
        self.space = space
        self.hyperparameters = hyperparameters
        self.preprocess_pipeline()
        self.preprocess_nodrop_pipeline()
        self.make_dataset_pipeline()

    def preprocess_pipeline(self):
        preprocess = Pipeline([
            ('drop_outlier', DropOutlier()),
            ('feature_engineering', FeatureEngineering())
        ])
        preprocessed_data = preprocess.fit_transform(df)
        self.output_data = preprocessed_data
        dropcolumn = Pipeline([
            ('drop_feature', DropFeature())
        ])
        preprocessed_data = dropcolumn.fit_transform(preprocessed_data)
        self.preprocessed_data = preprocessed_data

    def preprocess_nodrop_pipeline(self):
        pipeline = Pipeline([
            ('drop_outlier', DropOutlier()),
            ('feature_engineering', FeatureEngineering())
        ])
        self.preprocessed_data_nodrop = pipeline.fit_transform(df)
        return self.preprocessed_data_nodrop

    def make_dataset_pipeline(self):
        self.make_train_test_pred(self.preprocessed_data)
        self.model_pipeline()

    def model_pipeline(self):
        cat = CatPipeline(now_timestamp=self.now_timestamp, churn_day=90, test_size=30, pred_size=7, foldnum=4, jump_day=30, hyperparameters=True)
        cat.fit(df)
        ela = ElaPipeline(now_timestamp=self.now_timestamp, churn_day=90, test_size=30, pred_size=7, foldnum=4, jump_day=30, hyperparameters=True)
        ela.fit(df)
        xgb = XGBPipeline(now_timestamp=self.now_timestamp, churn_day=90, test_size=30, pred_size=7, foldnum=4, jump_day=30, hyperparameters=True)
        xgb.fit(df)
        self.es_tuning = pd.DataFrame({'cat': cat.test_pred[:, 1],
                                       'ela': ela.train_pred[:, 1],
                                       'xgb': xgb.test_pred[:, 1],
                                       'Churn': self.test['Churn']})
        self.es_final = pd.DataFrame({'cat': cat.final_test_pred[:, 1],
                                      'ela': ela.train_pred_r[:, 1],
                                      'xgb': xgb.final_test_pred[:, 1]})

    def return_score(self, y_test, y_pred, print_score=False, print_text=''):
        accuracy = accuracy_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        if print_score:
            print(print_text + f" Accuracy: {accuracy:.4f}")
            print(print_text + f" AUC: {auc:.4f}")
            print(print_text + f" F1-score: {f1:.4f}")
            print(print_text + f" Precision: {precision:.4f}")
        return accuracy, auc, f1, precision

    def make_train_test_pred(self, data):
        test_timestamp = pd.Timestamp(self.now_timestamp) - relativedelta(days=self.churn_day)
        train_timestamp = pd.Timestamp(test_timestamp) - relativedelta(days=self.churn_day)
        train = data[data['InvoiceDate'] <= train_timestamp].sort_values(by='InvoiceDate')
        test = data[(data['InvoiceDate'] > (test_timestamp - relativedelta(days=self.test_size))) &
                    (data['InvoiceDate'] <= test_timestamp)].sort_values(by='InvoiceDate')
        pred = data[(data['InvoiceDate'] > pd.Timestamp(self.now_timestamp) - relativedelta(days=self.pred_size)) &
                    (data['InvoiceDate'] <= pd.Timestamp(self.now_timestamp))].sort_values(by='InvoiceDate')
        final_train = data[data['InvoiceDate'] <= test_timestamp].sort_values(by='InvoiceDate')
        self.train = train.reset_index(drop=True)
        self.test = test.reset_index(drop=True)
        self.pred = pred.reset_index(drop=True)
        self.final_train = final_train.reset_index(drop=True)

    def make_output(self):
        output_data = self.output_data[['InvoiceDate', 'CustomerID']]
        test_timestamp = pd.Timestamp(self.now_timestamp) - relativedelta(days=self.churn_day)
        pred = output_data[(output_data['InvoiceDate'] > pd.Timestamp(self.now_timestamp) - relativedelta(days=self.pred_size)) &
                           (output_data['InvoiceDate'] <= pd.Timestamp(self.now_timestamp))].sort_values(by='InvoiceDate').reset_index(drop=True)
        pred['Churn'] = self.final_pred
        self.output = pred

    def tuning(self):
        data = self.es_tuning
        train, test = train_test_split(data, test_size=0.2, random_state=0)
        train, valid = train_test_split(train, test_size=0.2, random_state=0)
        X_train = train.drop('Churn', axis=1)
        y_train = train.Churn
        X_test = valid.drop('Churn', axis=1)
        y_test = valid.Churn

        def hyperparameter_tuning(space):
            model = LGBMClassifier(n_jobs=-1, early_stopping_rounds=None, **space,
                                   random_state=0,
                                   verbose=-1)
            model.fit(X_train, y_train)
            pred = model.predict(X_test)
            score = f1_score(y_test, pred)
            return {'loss': -score, 'status': STATUS_OK, 'model': model}

        trials = Trials()
        best = fmin(fn=hyperparameter_tuning,
                    space=self.space,
                    algo=tpe.suggest,
                    max_evals=200,
                    trials=trials,
                    rstate=np.random.default_rng(seed=0))
        self.hyperparameters = best

    def valid(self):
        if self.hyperparameters:
            self.tuning()
        data = self.es_tuning
        train, test = train_test_split(data, test_size=0.2, random_state=0)
        model = LGBMClassifier(**self.hyperparameters, random_state=0, verbose=-1)
        model.fit(train.drop('Churn', axis=1), train.Churn)
        pred = model.predict(test.drop('Churn', axis=1))
        self.return_score(test['Churn'], pred, print_score=True, print_text='Ensemble tuning score')

    def fit(self):
        if self.hyperparameters:
            self.tuning()
        final_model = LGBMClassifier(**self.hyperparameters, random_state=0, verbose=-1)
        final_model.fit(self.es_tuning.drop('Churn', axis=1), self.es_tuning.Churn)
        self.final_model = final_model

    def predict(self):
        final_pred = self.final_model.predict_proba(self.es_final)[:, 1]
        self.final_pred = final_pred
        self.make_output()
        return self.output